In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import operator




def wrap_ttest(df, label_column, comparison_columns=None, alpha=.05, return_all=False, correction_method='bonferroni', mincount=3, pval_return_corrected=True):
    try:
        '''Verify precondition that label column exists and has exactly 2 unique values'''
        label_values = df[label_column].unique()
        if len(label_values) != 2:
            print("Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.")
            return None
        
        '''Partition dataframe into two sets, one for each of the two unique values from the label column'''
        partition1 = df.loc[df[label_column] == label_values[0]]
        partition2 = df.loc[df[label_column] == label_values[1]]

        '''If no comparison columns specified, use all columns except the specified labed column'''
        if not comparison_columns:
            comparison_columns = list(df.columns)
            comparison_columns.remove(label_column)

        '''Determine the number of real valued columns on which we will do t-tests'''
        number_of_comparisons = len(comparison_columns)

        '''Store comparisons and p-values in two arrays'''
        comparisons = []
        pvals = []
        
        '''Loop through each comparison column, perform the t-test, and record the p-val'''
        for column in comparison_columns:
            if len(partition1[column].dropna(axis=0)) <= mincount:
                continue
            elif len(partition2[column].dropna(axis=0)) <= mincount:
                continue
            else:
                stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
                comparisons.append(column)
                pvals.append(pval)
            
        '''Correct for multiple testing to determine if each comparison meets the new cutoff'''
        results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=alpha, method=correction_method)
        reject = results[0]

        '''Format results in a pandas dataframe'''
        results_df = pd.DataFrame(columns=['Comparison','P_Value'])

        '''If return all, add all comparisons and p-values to dataframe'''
        if return_all:
            results_df['Comparison'] = comparisons
            results_df['P_Value'] = pvals

            '''Else only add significant comparisons'''
        else:
            for i in range(0, len(reject)):
                if reject[i]:
                    if pval_return_corrected:
                        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':results[1][i]}, ignore_index=True)
                    else:
                        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


        '''Sort dataframe by ascending p-value'''
        results_df = results_df.sort_values(by='P_Value', ascending=True)
        results_df = results_df.reset_index(drop=True)

        '''If results df is not empty, return it, else return None'''
        if len(results_df) > 0:
            return results_df
        else:
            print("Empty")
            return results[1]


    except:
        print("Incorrectly Formatted Dataframe!")
        return results[1]

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

cptac warning: Your version of cptac (0.8.0) is out-of-date. Latest is 0.8.1. Please run 'pip install --upgrade cptac' to update it. (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 10)


# Load cancer data

In [3]:
en = cptac.Endometrial()
br = cptac.Brca()
cl = cptac.Colon()

In [4]:
gene = 'PIK3CA'

# Endometrial

## Filter out hotspot mutations
Hotspots are:
E542K
E545K
H1047R


#### Get the mutation type, and proteomics for PIK3CA

In [5]:
proteomics = en.get_proteomics(tissue_type="tumor")
mut_type = en.get_genotype_all_vars("PIK3CA")

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


#### Join mutation type and proteomics together

In [6]:
joined = mut_type.join(proteomics)

In [7]:
joined

Name,Mutation,Location,Mutation_Status,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Missense_Mutation,p.E545K,Single_mutation,-1.180,-0.8630,-0.8020,0.2220,0.2560,0.6650,1.2800,...,-0.08770,NaN,0.02290,0.1090,NaN,-0.332,-0.43300,-1.020,-0.12300,-0.0859
C3L-00008,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-0.685,-1.0700,-0.6840,0.9840,0.1350,0.3340,1.3000,...,-0.03560,NaN,0.36300,1.0700,0.7370,-0.564,-0.00461,-1.130,-0.07570,-0.4730
C3L-00032,Missense_Mutation,p.E545K,Single_mutation,-0.528,-1.3200,0.4350,NaN,-0.2400,1.0400,-0.0213,...,0.00112,-0.1450,0.01050,-0.1160,NaN,0.151,-0.07400,-0.540,0.32000,-0.4190
C3L-00090,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-1.670,-1.1900,-0.4430,0.2430,-0.0993,0.7570,0.7400,...,0.07250,-0.0552,-0.07140,0.0933,0.1560,-0.398,-0.07520,-0.797,-0.03010,-0.4670
C3L-00098,Amplification,Amplification,Single_mutation,-0.374,-0.0206,-0.5370,0.3110,0.3750,0.0131,-1.1000,...,-0.17600,NaN,-1.22000,-0.5620,0.9370,-0.646,0.20700,-1.850,-0.17600,0.0513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-01520,Missense_Mutation,p.E726K,Single_mutation,-1.070,-0.7120,0.0462,-0.0471,0.2270,1.3500,1.2100,...,-0.06990,-0.4010,0.55700,0.8270,0.3480,-0.187,-0.10700,-0.830,0.06200,-0.5280
C3N-01521,Amplification,Amplification,Single_mutation,-1.280,-0.7360,-0.5520,0.1140,0.2730,1.0700,0.8500,...,0.13100,NaN,-0.00364,0.7630,0.0784,-0.203,0.01320,-1.200,-0.35700,0.0299
C3N-01537,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-0.290,-0.3200,3.1700,-0.9070,0.0317,-0.0425,NaN,...,-0.17900,-0.5210,0.05230,0.5150,0.5850,0.138,-0.19000,-0.966,-0.00627,-0.2490


#### Filter for missense and wildtyoe

In [8]:
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
missense = joined.loc[joined.Mutation == "Missense_Mutation"]

endo_both = missense.append(wildtype)

#### Drop unncecesary columns
The resulting dataframe is what we will be working with from this point forward. It contains the mutations type and the proteomics for each gene.

In [9]:
endo_prot_and_mutations = endo_both.drop(columns = ["Mutation_Status", "Location"])
endo_prot_and_mutations

# len(endo_prot_and_mutations.loc[endo_prot_and_mutations['Mutation']=="Missense_Mutation"])

Name,Mutation,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Missense_Mutation,-1.180,-0.8630,-0.8020,0.222,0.2560,0.6650,1.2800,-0.3390,0.4120,...,-0.08770,NaN,0.0229,0.109,NaN,-0.332,-0.4330,-1.020,-0.12300,-0.0859
C3L-00032,Missense_Mutation,-0.528,-1.3200,0.4350,NaN,-0.2400,1.0400,-0.0213,-0.0479,0.4190,...,0.00112,-0.1450,0.0105,-0.116,NaN,0.151,-0.0740,-0.540,0.32000,-0.4190
C3L-00139,Missense_Mutation,-0.467,0.3700,-0.3390,NaN,0.4340,0.0358,-0.1750,0.1810,0.1160,...,-0.67500,0.2390,0.1400,1.070,0.6070,0.486,0.1690,-0.632,-0.20300,-0.0685
C3L-00143,Missense_Mutation,-1.120,-1.3100,0.9120,0.418,-0.0768,0.8460,-0.1210,NaN,-0.3110,...,-0.00212,NaN,-1.1900,-1.270,-1.2700,-0.222,-0.3200,-0.620,0.36300,-0.4630
C3L-00156,Missense_Mutation,-0.232,-0.0910,0.6180,1.630,-0.2500,1.3300,-1.7200,-0.8890,0.6330,...,0.06520,-0.2360,0.5010,0.352,0.0839,-0.428,-0.2120,-1.340,0.31700,-0.9430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-01211,Wildtype_Tumor,-0.636,-0.3710,0.9730,NaN,0.6370,-0.1660,-0.1220,0.2320,0.3790,...,-0.41600,-0.7510,0.1790,0.658,0.4700,0.270,-0.1480,-0.139,-0.02990,0.1310
C3N-01217,Wildtype_Tumor,-0.743,-1.3700,NaN,NaN,-0.1930,0.6940,0.3380,0.0989,-0.0137,...,0.18100,NaN,0.1960,0.274,NaN,-0.501,-0.0468,0.653,-0.04640,-0.1600
C3N-01219,Wildtype_Tumor,-0.295,-0.0589,0.4900,-0.109,-0.0495,0.0374,0.1020,-0.3210,0.1830,...,-0.29000,0.0457,0.2460,0.589,0.3930,-0.199,-0.1590,-1.090,0.06270,-0.3790


## Run T-test and difference of Median for each cancer type

#### Get the median proteomic expression among all samples, for each gene.
This is done for missense mutations and wildtype separately


In [10]:
missense = endo_prot_and_mutations[endo_prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = endo_prot_and_mutations[endo_prot_and_mutations.Mutation == "Wildtype_Tumor"] #get all wildtype mutations
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype

#### Get the medain difference between missense and wildtype for each gene.
This is done by (for each gene) subtracting the median of the missense mutations from the median of the wildtype.
This means that if the difference is positive the proteomics of the wildtype have higher expression than the missense mutation. If the difference is negative then the missense mutation has a higher expression than the wildtype.

In [11]:
endo_d = {}

for gene in endo_prot_and_mutations:
    if gene == "Mutation": continue
    dif_endo =  missense_med[gene] - wt_med[gene]
    endo_d[gene] = dif_endo

median_diff = pd.DataFrame.from_dict(endo_d, orient='index', columns=['Difference_In_Median'])
median_diff.head()

,Difference_In_Median
A1BG,0.07100
A2M,0.10300
A2ML1,0.47135
A4GALT,0.22545
AAAS,0.07410


#### Do a t-test for every gene.

In [12]:
genes = list(endo_prot_and_mutations.columns[1:])
n = wrap_ttest(endo_prot_and_mutations, 'Mutation', correction_method="fdr_bh", mincount=15)
min(n)

Empty


0.9999312004874984

In [13]:
len(endo_prot_and_mutations.loc[endo_prot_and_mutations['Mutation']=="Missense_Mutation"])

43

In [15]:
# len(brca_prot_and_mutations.loc[brca_prot_and_mutations['Mutation']=="Missense_Mutation"])

In [16]:
# len(col_prot_and_mutations.loc[col_prot_and_mutations['Mutation']=="Missense_Mutation"])

#### Join difference in median and t-test p value into the same dataframe

In [17]:
# # Step 4 join median and p value together
# endo_df = median_diff.join(t_test)
# endo_df.head()

# BRCA

In [18]:
proteomics = br.get_proteomics(tissue_type="tumor")
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
no_duplicates = proteomics[proteomics.columns[~fil]]
# no_duplicates

In [19]:
#DEALING WITH DUPLICATES
#flatten the multiindex
duplicates = cptac.utils.reduce_multiindex(duplicates, flatten=True)

# We need to figure which of databaseID we want to use, for each protein
proteomics = br.get_proteomics(tissue_type="tumor")
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
duplicate_gene_names = list(set(duplicates.columns.get_level_values('Name'))) #get a list of genes names that are duplicated
duplicates = cptac.utils.reduce_multiindex(duplicates, flatten=True)
duplicates

#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest

mut_type = br.get_genotype_all_vars("PIK3CA")
joined = mut_type.join(duplicates)

#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]


#join wildtype proteomics to missense mutation proteomics 
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
missense = joined.loc[joined.Mutation == "Missense_Mutation"]
brca_both = missense.append(wildtype)

#drop unncessary columns
test_prot_and_mutations = brca_both.drop(columns = ["Mutation_Status", "Location"])


#step 3 do a t test for every gene and report the p-value
cols = list(test_prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(test_prot_and_mutations, 'Mutation', cols, correction_method= "fdr_bh", return_all=True)

# t_test

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


In [20]:
#for each duplicated gene name select the one with the min p val and append[ to no_duplicat df
selected_dup = []
for name in duplicate_gene_names:
    gene_duplicate = t_test[t_test.Comparison.str.contains(name)]
    #select the one with the min p value for t test
    min_row = gene_duplicate.loc[gene_duplicate.P_Value == gene_duplicate.P_Value.min()]
    name = min_row['Comparison'].values
    selected_dup.append(name)

selected= []


no_dup = cptac.utils.reduce_multiindex(no_duplicates, flatten=True)
no_dup.columns
for name in no_dup.columns:
    selected.append(name)

In [21]:
#use the list of selected genes to filter dataframe
proteomics = br.get_proteomics(tissue_type="tumor")
proteomics = cptac.utils.reduce_multiindex(proteomics, flatten=True)
selected_prot = proteomics[proteomics.columns.intersection(selected)]

#get mutation_tyoe
mut_type = br.get_genotype_all_vars('PIK3CA')

#join proteomics and mutation type
joined = mut_type.join(selected_prot)


#join wildtype proteomics to missense mutation proteomics 
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
missense = joined.loc[joined.Mutation == "Missense_Mutation"]
brca_both = missense.append(wildtype)


#drop unncessary columns
brca_prot_and_mutations = brca_both.drop(columns = ["Mutation_Status", "Location"])
#drop databaseID identifier
split = brca_prot_and_mutations.columns.str.split('_', 1).str[0]
brca_prot_and_mutations.columns=split

#remove duplicate columns
brca_prot_and_mutations = brca_prot_and_mutations.loc[:,~brca_prot_and_mutations.columns.duplicated()]
brca_prot_and_mutations

len(brca_prot_and_mutations.loc[brca_prot_and_mutations["Mutation"]== "Missense_Mutation"])

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


33

In [22]:
#now run ttest
n = wrap_ttest(brca_prot_and_mutations, 'Mutation', correction_method= "fdr_bh", pval_return_corrected=True, alpha=.3, mincount=20)
n

,Comparison,P_Value
0,BCL2L13,0.239758
1,HSPE1-MOB4,0.239758


# Colon

In [23]:
#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
proteomics = cl.get_proteomics(tissue_type="tumor")

prot_list = list(proteomics.columns)

mut_type = cl.get_genotype_all_vars("PIK3CA")


joined = mut_type.join(proteomics)


#join wildtype proteomics to missense mutation proteomics 
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
missense = joined.loc[joined.Mutation == 'nonsynonymous SNV']
colon_both = missense.append(wildtype)

#drop unncessary columns
col_prot_and_mutations = colon_both.drop(columns = ["Mutation_Status", "Location"])

col_prot_and_mutations["Mutation"] = col_prot_and_mutations['Mutation'].replace(['nonsynonymous SNV'], 'Missense_Mutation')
col_prot_and_mutations

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


Name,Mutation,A1BG,A1CF,A2M,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,...,ZNHIT6,ZNRD1,ZNRF2,ZPR1,ZRANB2,ZW10,ZWILCH,ZWINT,ZYX,ZZEF1
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01CO022,Missense_Mutation,0.906,-0.778,1.4500,0.359,0.2190,0.7710,-0.0471,-0.9160,0.4770,...,NaN,NaN,NaN,0.1940,0.1740,0.2990,NaN,NaN,-0.6200,-0.7180
05CO003,Missense_Mutation,-0.692,-1.220,-0.4380,0.828,-0.0137,0.2960,-0.0302,-1.2100,0.1550,...,NaN,NaN,-1.1400,0.1040,-0.1330,-0.1020,-1.0800,NaN,-0.0430,-0.2430
05CO006,Missense_Mutation,-1.190,-1.580,-0.6300,0.474,0.4050,0.2790,0.0172,-0.7120,-0.0105,...,NaN,NaN,-0.5520,-0.0722,0.3080,0.1620,-0.2670,NaN,-0.7660,-0.1950
05CO028,Missense_Mutation,-1.340,0.343,-1.6600,0.504,0.1620,0.3190,-0.2980,-0.2150,-0.1050,...,NaN,NaN,-0.0595,0.1690,0.1450,0.1970,-0.3990,NaN,-0.7540,-0.4230
05CO032,Missense_Mutation,-1.220,-0.575,-0.5960,0.299,-0.0254,0.7640,0.1040,-0.6940,0.3560,...,0.0105,NaN,-0.2850,-0.1480,0.0751,0.0503,0.2300,NaN,-0.5900,-0.0644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20CO001,Wildtype_Tumor,0.629,-0.068,0.0734,-0.654,0.2130,0.1550,0.5650,-0.0406,-0.1540,...,NaN,NaN,-0.1340,0.4150,-0.0765,-0.0385,NaN,NaN,0.4260,0.5190
20CO003,Wildtype_Tumor,-0.123,-1.070,-0.4520,0.266,-0.2280,0.0484,0.1710,0.7850,0.3000,...,NaN,NaN,NaN,-0.0415,0.0830,-0.0374,-0.1320,NaN,0.6810,-0.2970
22CO004,Wildtype_Tumor,-1.280,0.518,-0.8720,0.258,0.4240,0.2980,-0.1500,-0.6580,0.0066,...,NaN,NaN,0.4490,-0.0616,0.3850,0.1160,-0.0437,NaN,-0.8840,0.0088


In [24]:
len(col_prot_and_mutations.loc[col_prot_and_mutations["Mutation"]=="Missense_Mutation"])
col_prot_and_mutations.loc[col_prot_and_mutations["Mutation"]=="Missense_Mutation"]

Name,Mutation,A1BG,A1CF,A2M,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,...,ZNHIT6,ZNRD1,ZNRF2,ZPR1,ZRANB2,ZW10,ZWILCH,ZWINT,ZYX,ZZEF1
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01CO022,Missense_Mutation,0.9060,-0.7780,1.450,0.3590,0.2190,0.7710,-0.0471,-0.9160,0.4770,...,NaN,NaN,NaN,0.1940,0.1740,0.2990,NaN,NaN,-0.6200,-0.7180
05CO003,Missense_Mutation,-0.6920,-1.2200,-0.438,0.8280,-0.0137,0.2960,-0.0302,-1.2100,0.1550,...,NaN,NaN,-1.1400,0.1040,-0.1330,-0.1020,-1.080,NaN,-0.0430,-0.2430
05CO006,Missense_Mutation,-1.1900,-1.5800,-0.630,0.4740,0.4050,0.2790,0.0172,-0.7120,-0.0105,...,NaN,NaN,-0.5520,-0.0722,0.3080,0.1620,-0.267,NaN,-0.7660,-0.1950
05CO028,Missense_Mutation,-1.3400,0.3430,-1.660,0.5040,0.1620,0.3190,-0.2980,-0.2150,-0.1050,...,NaN,NaN,-0.0595,0.1690,0.1450,0.1970,-0.399,NaN,-0.7540,-0.4230
05CO032,Missense_Mutation,-1.2200,-0.5750,-0.596,0.2990,-0.0254,0.7640,0.1040,-0.6940,0.3560,...,0.0105,NaN,-0.2850,-0.1480,0.0751,0.0503,0.230,NaN,-0.5900,-0.0644
05CO050,Missense_Mutation,-1.2800,-0.2830,-0.869,0.8450,0.1110,0.9930,-0.0119,-0.7860,0.4540,...,NaN,NaN,NaN,-0.3500,0.2510,0.4000,0.417,0.0868,-1.0000,-0.2040
09CO005,Missense_Mutation,-1.4900,-0.2000,-1.280,-0.1220,0.2730,0.2230,0.3590,-1.1200,-0.0215,...,NaN,NaN,-0.2660,0.0078,0.5310,-0.3550,-0.640,NaN,-0.4300,-0.3340
11CO018,Missense_Mutation,-0.2400,-1.0800,-0.897,0.1770,0.5380,0.2650,-0.1210,-0.4080,-0.3480,...,NaN,NaN,-0.2150,0.1000,0.3080,0.0626,0.801,NaN,-0.2240,-0.2080
11CO027,Missense_Mutation,-1.0900,0.2360,-1.040,0.3070,0.0530,0.4660,-0.0733,-0.0244,0.8510,...,NaN,NaN,NaN,0.3930,0.7610,0.3730,NaN,NaN,0.0261,-0.1030


In [25]:
#get the difference in medians
missense = col_prot_and_mutations[col_prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = col_prot_and_mutations[col_prot_and_mutations.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


colon_d = {}

for prot in proteomics:
    dif_colon = missense_med[prot] - wt_med[prot]
    colon_d[prot] = dif_colon

median_diff = pd.DataFrame.from_dict(colon_d, orient='index', columns=['Difference_In_Median'])

In [26]:
#do a t test for every gene and report the p-value
cols = list(col_prot_and_mutations.columns[1:])
n = wrap_ttest(col_prot_and_mutations, 'Mutation', cols, correction_method= "fdr_bh", mincount=15)
# t_test = t_test.set_index('Comparison')
min(n)

Empty


0.313669112223768

In [27]:
# t_test.head(30)

In [28]:
#join median and p value together
colon_df = median_diff.join(t_test)
colon_df.head()

,Difference_In_Median,Comparison,P_Value
A1BG,-0.1530,NaN,NaN
A1CF,-0.3361,NaN,NaN
A2M,-0.0710,NaN,NaN
AAAS,0.0440,NaN,NaN
AACS,0.0370,NaN,NaN


# Combine Endo, Brca, and Colon into one table

#### Combine brca and endo

In [29]:
brca_df=brca_df.rename_axis(None)
brca_endo = brca_df.join(endo_df, lsuffix='_Brca', rsuffix='_Endo')
brca_endo.head()

NameError: name 'brca_df' is not defined

#### Combine colon

In [ ]:
final_table = brca_endo.join(colon_df, rsuffix='_colon')
final_table.head()

#### Only rows that contain a significant pvalue

In [ ]:
significant = final_table.loc[(final_table['P_Value_Brca'] <= .05) |
                (final_table['P_Value_Endo'] <= .05) |
                (final_table['P_Value'] <= .05)]
significant.head()

In [ ]:
significant.to_csv("~/WhenMutationsDontMatter/PIK3CA/csv_files/proteomics.csv")